In [2]:
import pandas as pd
import numpy as np

# 1. Dataset Specification: SisFall
**Project:** Hybrid Privacy-Preserving Federated Learning (HPPFL) for Rural-Urban IoMT  
**Dataset Source:** SisFall (Fall and Movement Dataset)  
**Sampling Rate:** 200 Hz  

The dataset consists of **9 raw data columns** collected simultaneously from three separate sensors located on a single device (worn at the waist). The distinct sensors provide "multi-view" data, capturing both high-sensitivity movements (like walking) and high-impact events (like falling).

### Sensor Schema

| Column Index | Variable Name | Sensor Hardware | Sensor Type | Axis | Description |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **0** | `ADXL_x` | **ADXL345** | Accelerometer | X | **High-Range Sensor.** Digital accelerometer with up to $\pm 16g$ range. Ideal for capturing high-impact peaks during falls that might saturate other sensors. |
| **1** | `ADXL_y` | **ADXL345** | Accelerometer | Y | |
| **2** | `ADXL_z` | **ADXL345** | Accelerometer | Z | |
| **3** | `ITG_x` | **ITG3200** | Gyroscope | X | **Rotation Sensor.** Measures angular velocity (spin) up to $\pm 2000^\circ/s$. Critical for distinguishing falls (rotational changes) from lying down. |
| **4** | `ITG_y` | **ITG3200** | Gyroscope | Y | |
| **5** | `ITG_z` | **ITG3200** | Gyroscope | Z | |
| **6** | `MMA_x` | **MMA8451Q** | Accelerometer | X | **High-Sensitivity Sensor.** Digital accelerometer with $\pm 8g$ range and 14-bit resolution. Captures finer details of Activities of Daily Living (ADL) like walking or sitting. |
| **7** | `MMA_y` | **MMA8451Q** | Accelerometer | Y | |
| **8** | `MMA_z` | **MMA8451Q** | Accelerometer | Z | |

In [10]:
# Define the column names based on the sensor hardware
col_names = [
    "ADXL_x", "ADXL_y", "ADXL_z",  # Sensor 1: ADXL345 Accelerometer
    "ITG_x",  "ITG_y",  "ITG_z",   # Sensor 2: ITG3200 Gyroscope
    "MMA_x",  "MMA_y",  "MMA_z"    # Sensor 3: MMA8451Q Accelerometer
]

# 2. Activity Labels (ADLs)
**Category:** Activities of Daily Living (ADL)  
**Code Prefix:** D (Daily)

The dataset distinguishes between 19 different types of daily activities. These serve as the "Negative" class (Non-Fall) in the classification task.

| Code | Activity Description | Trials | Duration |
| :--- | :--- | :--- | :--- |
| **D01** | Walking slowly | 1 | 100s |
| **D02** | Walking quickly | 1 | 100s |
| **D03** | Jogging slowly | 1 | 100s |
| **D04** | Jogging quickly | 1 | 100s |
| **D05** | Walking upstairs and downstairs slowly | 5 | 25s |
| **D06** | Walking upstairs and downstairs quickly | 5 | 25s |
| **D07** | Slowly sit in a half height chair, wait, and up slowly | 5 | 12s |
| **D08** | Quickly sit in a half height chair, wait, and up quickly | 5 | 12s |
| **D09** | Slowly sit in a low height chair, wait, and up slowly | 5 | 12s |
| **D10** | Quickly sit in a low height chair, wait, and up quickly | 5 | 12s |
| **D11** | Sitting a moment, trying to get up, and collapse into chair | 5 | 12s |
| **D12** | Sitting a moment, lying slowly, wait, and sit again | 5 | 12s |
| **D13** | Sitting a moment, lying quickly, wait, and sit again | 5 | 12s |
| **D14** | Changing position (back to lateral), wait, and back again | 5 | 12s |
| **D15** | Standing, slowly bending at knees, and getting up | 5 | 12s |
| **D16** | Standing, slowly bending without bending knees, and up | 5 | 12s |
| **D17** | Standing, get into a car, remain seated, and get out | 5 | 25s |
| **D18** | Stumble while walking | 5 | 12s |
| **D19** | Gently jump without falling (reach high object) | 5 | 12s |

# 3. Fall Labels (Falls)
**Category:** Fall Events  
**Code Prefix:** F (Fall)

These 15 activities represent various fall scenarios (slips, trips, fainting). In a Binary Classification task, these constitute the "Positive" class (Fall Detected).

| Code | Activity Description | Trials | Duration |
| :--- | :--- | :--- | :--- |
| **F01** | Fall forward while walking (slip) | 5 | 15s |
| **F02** | Fall backward while walking (slip) | 5 | 15s |
| **F03** | Lateral fall while walking (slip) | 5 | 15s |
| **F04** | Fall forward while walking (trip) | 5 | 15s |
| **F05** | Fall forward while jogging (trip) | 5 | 15s |
| **F06** | Vertical fall while walking (fainting) | 5 | 15s |
| **F07** | Fall while walking, using hands to dampen impact (fainting) | 5 | 15s |
| **F08** | Fall forward when trying to get up | 5 | 15s |
| **F09** | Lateral fall when trying to get up | 5 | 15s |
| **F10** | Fall forward when trying to sit down | 5 | 15s |
| **F11** | Fall backward when trying to sit down | 5 | 15s |
| **F12** | Lateral fall when trying to sit down | 5 | 15s |
| **F13** | Fall forward while sitting (fainting/asleep) | 5 | 15s |
| **F14** | Fall backward while sitting (fainting/asleep) | 5 | 15s |
| **F15** | Lateral fall while sitting (fainting/asleep) | 5 | 15s |

In [11]:
# Full SisFall Activity Map (D + F)
activity_map = {
    # --- ADLs (Activities of Daily Living) ---
    'D01': 'Walking slowly',
    'D02': 'Walking quickly',
    'D03': 'Jogging slowly',
    'D04': 'Jogging quickly',
    'D05': 'Upstairs/downstairs slowly',
    'D06': 'Upstairs/downstairs quickly',
    'D07': 'Sit half-height slowly',
    'D08': 'Sit half-height quickly',
    'D09': 'Sit low-height slowly',
    'D10': 'Sit low-height quickly',
    'D11': 'Sit, try up, collapse',
    'D12': 'Sit, lie slowly, sit',
    'D13': 'Sit, lie quickly, sit',
    'D14': 'Change pos (back/lateral)',
    'D15': 'Stand, bend knees, up',
    'D16': 'Stand, bend no-knees, up',
    'D17': 'Car entry/exit',
    'D18': 'Stumble walking',
    'D19': 'Gentle jump',
    
    # --- Falls ---
    'F01': 'Fall fwd walking (slip)',
    'F02': 'Fall bwd walking (slip)',
    'F03': 'Fall lat walking (slip)',
    'F04': 'Fall fwd walking (trip)',
    'F05': 'Fall fwd jogging (trip)',
    'F06': 'Vertical fall (fainting)',
    'F07': 'Fall w/ dampening (fainting)',
    'F08': 'Fall fwd trying to get up',
    'F09': 'Fall lat trying to get up',
    'F10': 'Fall fwd trying to sit',
    'F11': 'Fall bwd trying to sit',
    'F12': 'Fall lat trying to sit',
    'F13': 'Fall fwd sitting (faint)',
    'F14': 'Fall bwd sitting (faint)',
    'F15': 'Fall lat sitting (faint)'
}

In [ ]:
def get_binary_label(filename):
    """
    Returns 1 if the activity is a FALL (F01-F15), 
    Returns 0 if the activity is an ADL (D01-D19).
    """
    code = filename.split('_')[0] # Extracts 'D01', 'F02', etc.
    
    if code.startswith('F'):
        return 1  # Fall
    else:
        return 0  # No Fall (ADL)

# Example Usage:
# label = get_binary_label("F03_SA01_R01.txt")
# print(label)  # Output: 1

# 4. Subject Demographics
**Total Subjects:** 38  
**Groups:** 
* **SA:** Young/Adult Subjects (SA01–SA23)
* **SE:** Elderly Subjects (SE01–SE15)

| Subject ID | Age | Height (cm) | Weight (kg) | Gender |
| :--- | :--- | :--- | :--- | :--- |
| **SA01** | 26|165 |53 |F |
| **SA02** |23 |176 |58.5 |M |
| **SA03** |19 |156 |48 |F |
| **SA04** |23 |170 |72 |M |
| **SA05** |22 |172 |69.5 |M |
| **SA06** |21 |169 |58 |M |
| **SA07** |21 |156 |63 |F |
| **SA08** |21 |149 |41.5 |F |
| **SA09** |24 |165 |64 |M |
| **SA10** |21 |177 |67 |M |
| **SA11** |19 |170 |80.5 |M |
| **SA12** |25 |153 |47 |F |
| **SA13** |22 |157 |55 |F |
| **SA14** |27 |160 |46 |F |
| **SA15** |25 |160 |52 |F |
| **SA16** |20 |169 |61 |F |
| **SA17** |23 |182 |75 |M |
| **SA18** |23 |181 |73 |M |
| **SA19** |30 |170 |76 |M |
| **SA20** |30 |150 |42 |F |
| **SA21** |30 |183 |68 |M |
| **SA22** |19 |158 |50.5 |F |
| **SA23** |24 |156 |48 |F |
| **SE01** |71 |171 |102 |M |
| **SE02** |75 |150 |57 |F |
| **SE03** |62 |150 |51 |F |
| **SE04** |63 |160 |59 |F |
| **SE05** |63 |165 |72 |M |
| **SE06** |60 |163 |79 |M |
| **SE07** |65 |168 |76 |M |
| **SE08** |68 |163 |72 |F |
| **SE09** |66 |167 |65 |M |
| **SE10** |64 |156 |66 |F |
| **SE11** |66 |169 |63 |F |
| **SE12** |69 |164 |56.5 |M |
| **SE13** |65 |171 |72.5 |M |
| **SE14** |67 |163 |58 |M |
| **SE15** |64 |150 |50 |F |

In [13]:
# Subject Demographic Data
# Usage: subject_data['SA01']['age'] returns the age

subject_data = {
    # --- Young/Adult Group (SA) ---
    'SA01': {'age': 26, 'height': 165, 'weight': 53, 'gender': 'F'},
    'SA02': {'age': 23, 'height': 176, 'weight': 58.5, 'gender': 'M'},
    'SA03': {'age': 19, 'height': 156, 'weight': 48, 'gender': 'F'},
    'SA04': {'age': 23, 'height': 170, 'weight': 72, 'gender': 'M'},
    'SA05': {'age': 22, 'height': 172, 'weight': 69.5, 'gender': 'M'},
    'SA06': {'age': 21, 'height': 169, 'weight': 58, 'gender': 'M'},
    'SA07': {'age': 21, 'height': 169, 'weight': 58, 'gender': 'F'},
    'SA08': {'age': 21, 'height': 156, 'weight': 63, 'gender': 'F'},
    'SA09': {'age': 24, 'height': 165, 'weight': 64, 'gender': 'M'},
    'SA10': {'age': 21, 'height': 177, 'weight': 67, 'gender': 'M'},
    'SA11': {'age': 19, 'height': 170, 'weight': 80.5, 'gender': 'M'},
    'SA12': {'age': 25, 'height': 153, 'weight': 47, 'gender': 'F'},
    'SA13': {'age': 22, 'height': 157, 'weight': 55, 'gender': 'F'},
    'SA14': {'age': 27, 'height': 160, 'weight': 46, 'gender': 'F'},
    'SA15': {'age': 25, 'height': 160, 'weight': 52, 'gender': 'F'},
    'SA16': {'age': 20, 'height': 169, 'weight': 61, 'gender': 'F'},
    'SA17': {'age': 23, 'height': 182, 'weight': 75, 'gender': 'M'},
    'SA18': {'age': 23, 'height': 181, 'weight': 73, 'gender': 'M'},
    'SA19': {'age': 30, 'height': 170, 'weight': 76, 'gender': 'M'},
    'SA20': {'age': 30, 'height': 150, 'weight': 42, 'gender': 'F'},
    'SA21': {'age': 30, 'height': 183, 'weight': 68, 'gender': 'M'},
    'SA22': {'age': 19, 'height': 158, 'weight': 50.5, 'gender': 'F'},
    'SA23': {'age': 24, 'height': 156, 'weight': 48, 'gender': 'F'},
    
    # --- Elderly Group (SE) ---
    'SE01': {'age': 71, 'height': 171, 'weight': 102, 'gender': 'M'},
    'SE02': {'age': 75, 'height': 150, 'weight': 57, 'gender': 'F'},
    'SE03': {'age': 62, 'height': 150, 'weight': 51, 'gender': 'F'},
    'SE04': {'age': 63, 'height': 160, 'weight': 59, 'gender': 'F'},
    'SE05': {'age': 63, 'height': 165, 'weight': 72, 'gender': 'M'},
    'SE06': {'age': 60, 'height': 163, 'weight': 79, 'gender': 'M'},
    'SE07': {'age': 65, 'height': 168, 'weight': 76, 'gender': 'M'},
    'SE08': {'age': 68, 'height': 163, 'weight': 72, 'gender': 'F'},
    'SE09': {'age': 66, 'height': 167, 'weight': 65, 'gender': 'M'},
    'SE10': {'age': 64, 'height': 156, 'weight': 66, 'gender': 'F'},
    'SE11': {'age': 66, 'height': 169, 'weight': 63, 'gender': 'F'},
    'SE12': {'age': 69, 'height': 164, 'weight': 56.5, 'gender': 'M'},
    'SE13': {'age': 65, 'height': 171, 'weight': 72.5, 'gender': 'M'},
    'SE14': {'age': 67, 'height': 163, 'weight': 58, 'gender': 'M'},
    'SE15': {'age': 64, 'height': 150, 'weight': 50, 'gender': 'F'},
}

In [ ]:
def get_is_elderly(filename):
    """
    Function 1: Binary Classification for FL Client Routing.
    Returns:
        1 if the subject is Elderly (Rural/SE)
        0 if the subject is Young (Urban/SA)
    """
    # Extract the middle part of 'D01_SA01_R01.txt'
    subject_id = filename.split('_')[1] 
        
    # Check if ID starts with 'SE' (Subject Elderly)
    if subject_id.startswith("SE"):
        return 1 # Subject elderly
    else:
        return 0 # Subject adult

def get_demographics(filename):
    """
    Function 2: Detailed Metadata for Analysis.
    Returns:
        Dictionary containing age, height, weight, gender.
    """
    subject_id = filename.split('_')[1]
        
    # Look up the ID in the master dictionary
    # Returns empty dict {} if ID is not found
    return subject_data.get(subject_id, {})

In [9]:
practice_df = pd.read_csv("../data/archive/SisFall_dataset/SA01/D01_SA01_R01.txt",header=None)
practice_df.head()

,0,1,2,3,4,5,6,7,8
0,17,-179,-99,-18,-504,-352,76,-697,-279;
1,15,-174,-90,-53,-568,-306,48,-675,-254;
2,1,-176,-81,-84,-613,-271,-2,-668,-221;
3,-10,-180,-77,-104,-647,-227,-34,-697,-175;
4,-21,-191,-63,-128,-675,-191,-74,-741,-133;
